<a href="https://colab.research.google.com/github/krishan74/covid19/blob/main/Wave2Vec_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May 25 06:31:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install jiwer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
%%capture
!apt install git-lfs

In [ ]:
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")

Downloading:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/869M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset timit_asr downloaded and prepared to /root/.cache/huggingface/datasets/timit_asr/clean/2.0.1/b11b576ddcccbcefa7c9f0c4e6c2a43756f3033adffe0fb686aa61043d0450ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

In [ ]:
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(timit["train"].remove_columns(["audio", "file"]), num_examples=10)

,text
0,She had your dark suit in greasy wash water all year.
1,She had your dark suit in greasy wash water all year.
2,His first glimpse of the ranch house across the brushy swells told him nothing.
3,Fights sprang up and were quickly squelched.
4,Don't ask me to carry an oily rag like that.
5,We'll serve rhubarb pie after Rachel's talk.
6,So we note approvingly a fresh sample of unanimity.
7,Each plane carried two five-hundred pound bombs.
8,Don't ask me to carry an oily rag like that.
9,They often go out in the evening.


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
timit = timit.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x7fa098dd6cb0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [ ]:
show_random_elements(timit["train"].remove_columns(["audio", "file"]))

,text
0,coconut cream pie makes a nice dessert
1,only the best players enjoy popularity
2,while waiting for chipper she crisscrossed the square many times
3,what is this large thing by the ironing board
4,honey he whispered
5,the sermon emphasized the need for affirmative action
6,would have been easy to identify as opium by its odor
7,don't ask me to carry an oily rag like that
8,why not try another club
9,how do oysters make pearls


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 2,
 "'": 21,
 'a': 10,
 'b': 23,
 'c': 6,
 'd': 0,
 'e': 26,
 'f': 18,
 'g': 1,
 'h': 9,
 'i': 17,
 'j': 20,
 'k': 24,
 'l': 8,
 'm': 13,
 'n': 27,
 'o': 5,
 'p': 25,
 'q': 12,
 'r': 16,
 's': 7,
 't': 3,
 'u': 19,
 'v': 11,
 'w': 4,
 'x': 14,
 'y': 22,
 'z': 15}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
repo_name = "wav2vec2-base-timit-demo-google-colab"

In [ ]:
#tokenizer.push_to_hub(repo_name)

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
timit["train"][0]["file"]

'/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR4/MMDM0/SI681.WAV'

In [ ]:
timit["train"][0]["audio"]

{'array': array([-2.1362305e-04,  6.1035156e-05,  3.0517578e-05, ...,
        -3.0517578e-05, -9.1552734e-05, -6.1035156e-05], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR4/MMDM0/SI681.WAV',
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

print(timit["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

the overweight charmer could slip poison into anyone's tea 


In [ ]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"])

Target text: the advertising verse of plymouth variety store never changes 
Input array shape: (68916,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
timit = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4)

In [ ]:
max_input_length_in_sec = 4.0
timit["train"] = timit["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'project_q.bias', 'project_q.weight', 'project_hid.bias', 'project_hid.weight', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
model.freeze_feature_encoder()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='./results',
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit["train"],
    eval_dataset=timit["test"],
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3978
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 14940


Step,Training Loss,Validation Loss,Wer
500,3.590500,2.131872,1.051685
1000,0.956700,0.554309,0.537179
1500,0.464600,0.459036,0.463993
2000,0.317900,0.423107,0.430708
2500,0.242300,0.404305,0.410241
3000,0.202400,0.457461,0.393426
3500,0.161200,0.449660,0.380746
4000,0.141100,0.413464,0.384260
4500,0.126800,0.458915,0.383778
5000,0.110900,0.453102,0.375508


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1680
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Feature extractor saved in ./results/checkpoint-500/preprocessor_config.json
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1680
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
M

TrainOutput(global_step=14940, training_loss=0.24354455091228766, metrics={'train_runtime': 6624.1664, 'train_samples_per_second': 18.016, 'train_steps_per_second': 2.255, 'total_flos': 3.0766610595969556e+18, 'train_loss': 0.24354455091228766, 'epoch': 30.0})

In [ ]:
#trainer.push_to_hub()

OSError: ignored

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")
```

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")

https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/preprocessor_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmppt41h5qm


Downloading:   0%|          | 0.00/215 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/preprocessor_config.json in cache at /root/.cache/huggingface/transformers/17fa07f6e2f88edf24c71037bed1b7cf0cf9acfef7d3353afb589689627542c1.0e3e6656f99a6f7b9eddd943463eb7f34363640fce9e87b047ebd50d4b112b50
creating metadata file for /root/.cache/huggingface/transformers/17fa07f6e2f88edf24c71037bed1b7cf0cf9acfef7d3353afb589689627542c1.0e3e6656f99a6f7b9eddd943463eb7f34363640fce9e87b047ebd50d4b112b50
loading feature extractor configuration file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/preprocessor_config.json from cache at /root/.cache/huggingface/transformers/17fa07f6e2f88edf24c71037bed1b7cf0cf9acfef7d3353afb589689627542c1.0e3e6656f99a6f7b9eddd943463eb7f34363640fce9e87b047ebd50d4b112b50
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_s

Downloading:   0%|          | 0.00/217 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/7bf2e07c1c7ed7e3d055d87aa6bd33c75e6883d98029ae3744fed6bea9e35095.59710b1a6a5501d31e746b6e464f5c44de3e55a58f80634196025936683a68a9
creating metadata file for /root/.cache/huggingface/transformers/7bf2e07c1c7ed7e3d055d87aa6bd33c75e6883d98029ae3744fed6bea9e35095.59710b1a6a5501d31e746b6e464f5c44de3e55a58f80634196025936683a68a9
https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/vocab.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpp63i263t


Downloading:   0%|          | 0.00/268 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/f739813380b0880ba302557f0f990f0f64e8efe81fcc91a4fce08206a91ee215.c44a036121683e19f1a033a1765d4344364cb4633a6cbf5cf51b153f2388abb1
creating metadata file for /root/.cache/huggingface/transformers/f739813380b0880ba302557f0f990f0f64e8efe81fcc91a4fce08206a91ee215.c44a036121683e19f1a033a1765d4344364cb4633a6cbf5cf51b153f2388abb1
https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpjhn2047l


Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/c239f3738252f8d8e83d9813ea82866aeec476c467a008db86229f4edc6490b0.a21d51735cf8667bcd610f057e88548d5d6a381401f6b4501a8bc6c1a9dc8498
creating metadata file for /root/.cache/huggingface/transformers/c239f3738252f8d8e83d9813ea82866aeec476c467a008db86229f4edc6490b0.a21d51735cf8667bcd610f057e88548d5d6a381401f6b4501a8bc6c1a9dc8498
loading file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/f739813380b0880ba302557f0f990f0f64e8efe81fcc91a4fce08206a91ee215.c44a036121683e19f1a033a1765d4344364cb4633a6cbf5cf51b153f2388abb1
loading file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/7bf2e07c1c7ed7e3d055d87aa6b

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab").cuda()

https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpaw2rhekt


Downloading:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/7606a293c66b1cba6bb9a342dcffe3831dc19a120cfeeb9e57cdf6ffb9630f3f.3202909be16e9c482257d0c4d35b4f4c2558cea585027c729eed803ba4bf0d8e
creating metadata file for /root/.cache/huggingface/transformers/7606a293c66b1cba6bb9a342dcffe3831dc19a120cfeeb9e57cdf6ffb9630f3f.3202909be16e9c482257d0c4d35b4f4c2558cea585027c729eed803ba4bf0d8e
loading configuration file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/7606a293c66b1cba6bb9a342dcffe3831dc19a120cfeeb9e57cdf6ffb9630f3f.3202909be16e9c482257d0c4d35b4f4c2558cea585027c729eed803ba4bf0d8e
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment":

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/666767bce47a87d50dfd1ceac759d5dc9db6eac8d185ae8f9a7dd388687c28c5.20bdf845b9896c1d0a213f03187b544f2a12b23d751b88c7b49a95d00bd946e1
creating metadata file for /root/.cache/huggingface/transformers/666767bce47a87d50dfd1ceac759d5dc9db6eac8d185ae8f9a7dd388687c28c5.20bdf845b9896c1d0a213f03187b544f2a12b23d751b88c7b49a95d00bd946e1
loading weights file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/666767bce47a87d50dfd1ceac759d5dc9db6eac8d185ae8f9a7dd388687c28c5.20bdf845b9896c1d0a213f03187b544f2a12b23d751b88c7b49a95d00bd946e1
All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

All the weights of Wav2Vec2ForCTC were initialized from the model checkpoint at patrickvonplaten/wav2vec2-base-timit-demo-

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [ ]:
results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

0ex [00:00, ?ex/s]

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 4.741


In [ ]:
show_random_elements(results)

,pred_str,text
0,young children should avoid exposure to contageous diseases,uzcpnsdhxakywpslhzcaksmvzxkswitzlcywsozsdzpomnxzclskxlwmlwls
1,art official intelligence is for real,myoxfxdxmasxpowaaxnwpdwsxlsfzysywmas
2,theire propes were two step latters a chair and a pame fen,ohwxystyztls wywso zslowtamkkwylsmsdhmxysmpksmstmagsfmps
3,if people were more generous there would be no need for wealfar,xfstwztaws wywsgzywsnwpwyzclsohwyws zcaksbwspzspwwksfzys wafmyws
4,the fish began to leep frantically on the surface of the smalleaku,ohwsfxlhsbwnmpsozsawmtsfympoxdmaauszpsohwslcyfmdwszfsohwslgmaasamrws
5,her rite hand aques whenever the baramatric presuer changes,hwysyxnhoshmpksmdhwls hwpwvwysohwsbmyzgwoyxdstywllcywsdhmpnwls
6,only lawyers loved milunears,zpausam uwylsazvwsgxaaxzpmxywls
7,the nearest synnegu may not be within walk in distance,ohwspwmywloslupmnzncwsgmuspzosbws xohxps marxpnskxlompdws
8,basket ball can be an entertaining sport,bmlrwobmaasdmpsbwsmpswpowyomxpxpnsltzyos
9,she had your dark suit in greasy wash water all year,lhwshmksuzcyskmyrslcxosxpsnywmlus mlhs mowysmaasuwmys


In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(timit["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] [PAD] [PAD] [PAD] t t h h e | | | b [PAD] [PAD] [PAD] u u n n g g [PAD] [PAD] l l l l [PAD] o o o | | w w a a s s | | [PAD] [PAD] [PAD] p l l [PAD] e s s s s [PAD] n n t t [PAD] l l l y y | | [PAD] s s s i i t t t [PAD] u u u u u [PAD] [PAD] [PAD] [PAD] a a t t [PAD] e e d d | | [PAD] n n e e a a r | | t t h e e | | s s h h [PAD] [PAD] [PAD] [PAD] o o r r r r | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
timit

NameError: ignored